In [1]:
!pip install pandas
!pip install sentence_transformers
!pip install sklearn

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity # 챗봇의 정확도 계산

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["학교가 어디인가요?", "저는 부산 소마고에 다니고 있습니다."]
# 두 문장을 숫자로 임베딩
embeddings = model.encode(sentences)

print(embeddings)

[[ 0.10478407 -0.19161403  0.03850418 ...  0.60394835  0.50577533
  -0.736189  ]
 [ 0.10454009  0.19116943 -0.9161043  ... -0.16573702  0.20779425
  -0.34834814]]


In [4]:
# 데이터셋 로드
df = pd.read_csv('Chatbot_data.csv')
df.head()

,구분,유저,챗봇,embedding,distance
0,입학홍보,부산소마고 페이스북,[ https://www.facebook.com/BusanSoftwareMeiste...,"[-0.41640186, 0.079589315, -0.5161981, 0.37378...",0.595668
1,입학홍보,부산소프트웨어마이스터고 페북,[ https://www.facebook.com/BusanSoftwareMeiste...,NaN,NaN
2,입학홍보,소마고 페북,[ https://www.facebook.com/BusanSoftwareMeiste...,NaN,NaN
3,입학홍보,부산소마고 인스타그램,[ https://www.instagram.com/bssm.hs/ ] 입니다!,"[-0.34022376, -0.339683, -0.53931576, 0.615103...",0.591826
4,입학홍보,부산소프트웨어마이스터고 인스타그램,[ https://www.instagram.com/bssm.hs/ ] 입니다!,NaN,NaN


In [5]:
df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

df.head()

,구분,유저,챗봇,embedding,distance
0,입학홍보,부산소마고 페이스북,[ https://www.facebook.com/BusanSoftwareMeiste...,"[-0.41640186, 0.079589315, -0.5161981, 0.37378...",0.595668
1,입학홍보,부산소프트웨어마이스터고 페북,[ https://www.facebook.com/BusanSoftwareMeiste...,"[-0.07906039, -0.12451441, -0.408753, 0.720848...",NaN
2,입학홍보,소마고 페북,[ https://www.facebook.com/BusanSoftwareMeiste...,"[0.11361439, -0.050491035, -0.04860738, 0.0593...",NaN
3,입학홍보,부산소마고 인스타그램,[ https://www.instagram.com/bssm.hs/ ] 입니다!,"[-0.34022376, -0.339683, -0.53931576, 0.615103...",0.591826
4,입학홍보,부산소프트웨어마이스터고 인스타그램,[ https://www.instagram.com/bssm.hs/ ] 입니다!,"[-0.19117956, -0.18577842, -0.4473932, 0.85997...",NaN


In [7]:
# 위에서 임베딩한 결과까지 csv 파일로 저장된다
df.to_csv('Chatbot_embedding_data.csv', index = False)

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [8]:
text = '부산 소마고 페이스북' # 입력할 텍스트
embedding = model.encode(text) # 텍스트를 인코딩해서 embedding에 넣는다

In [9]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
# 사용자가 encode한 text와 가장 유사한 문장을 찾기 위해 embedding을 비교해 찾는다
df.head()   

,구분,유저,챗봇,embedding,distance
0,입학홍보,부산소마고 페이스북,[ https://www.facebook.com/BusanSoftwareMeiste...,"[-0.41640186, 0.079589315, -0.5161981, 0.37378...",0.904900
1,입학홍보,부산소프트웨어마이스터고 페북,[ https://www.facebook.com/BusanSoftwareMeiste...,"[-0.07906039, -0.12451441, -0.408753, 0.720848...",0.527573
2,입학홍보,소마고 페북,[ https://www.facebook.com/BusanSoftwareMeiste...,"[0.11361439, -0.050491035, -0.04860738, 0.0593...",0.830662
3,입학홍보,부산소마고 인스타그램,[ https://www.instagram.com/bssm.hs/ ] 입니다!,"[-0.34022376, -0.339683, -0.53931576, 0.615103...",0.823811
4,입학홍보,부산소프트웨어마이스터고 인스타그램,[ https://www.instagram.com/bssm.hs/ ] 입니다!,"[-0.19117956, -0.18577842, -0.4473932, 0.85997...",0.494270


In [ ]:
answer = df.loc[df['distance'].idxmax()] # distance의 최댓값 = 가장 유사함!

print(answer['구분'], "관련 질문이군요!")
print(answer['유저'] + "은,", answer['챗봇'], "입니다!")
# print(answer['챗봇'])
print("(답변 유사도 : ", answer['distance'], ")")

입학홍보 관련 질문이군요!
부산소마고 페이스북은, https://www.facebook.com/BusanSoftwareMeisterHighschool/ 입니다!
(답변 유사도 :  0.9048997163772583 )
